In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from pprint import pprint
from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent
from typing import Dict, Any, Optional
from tavily import TavilyClient
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.checkpoint.memory import InMemorySaver
from langchain.chat_models import init_chat_model
from langgraph.types import Command
from langchain.messages import HumanMessage, ToolMessage
from langchain.agents import AgentState

## LLM Model

In [3]:
model = init_chat_model(model="gpt-5-nano")

## Tools

### Web Search

In [4]:
tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)


web_search.invoke("today's date")

{'query': "today's date",
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.datetoday.net/',
   'title': "What is the date today? Today's Date",
   'content': 'Today\'s Date is Sun Feb 15 2026. About Date Today. The term "Date Today" refers to the current calendar date as determined by a system\'s internal clock',
   'score': 0.9981178,
   'raw_content': None},
  {'url': 'https://www.inchcalculator.com/what-is-todays-date/',
   'title': "What Is Today's Date? - Inch Calculator",
   'content': "Today, February 15th , is day 46 of 365 total days in 2026. What is Today's Date in Numbers? Today's date in numbers is: MM-DD-YYYY: 02-15-2026; DD-MM-YYYY",
   'score': 0.98934746,
   'raw_content': None},
  {'url': 'https://www.calendardate.com/todays.htm',
   'title': "Today's Date - CalendarDate.com",
   'content': "Details about today's date with count of days, weeks, and months, Sun and Moon cycles, Zodiac signs and holidays.",
   'score': 0.985

### Flight Search

In [5]:
travel_server_mcp = MultiServerMCPClient(
    {
        "mcpServers": {
            "transport": "streamable_http",
            "url": "https://mcp.kiwi.com"
            }
        }
)
travel_tools = await travel_server_mcp.get_tools()

travel_tools

[StructuredTool(name='search-flight', description='\n# Search for a flight\n\n## Description\n\nUses the Kiwi API to search for available flights between two locations on a specific date.\n\n## How it works\n\nThe tool will:\n1. Search for matching locations to resolve airport codes\n2. Find available flights for the specified route and date range\n\n## Method\n\nCall this tool whenever a user wants to search for flights, regardless of whether they provided exact airport codes or just city names.\n\nYou should display the returned results in a markdown table format: Group the results by price (those who are the cheapest), duration (those who are the shortest, i.e. have the smallest \'totalDurationInSeconds\') and the rest (those that could still be interesting).\n\nAlways display for each flight in order:\n  - In the 1st column: The departure and arrival airports, including layovers (e.g. "Paris CDG → Barcelona BCN → Lisbon LIS")\n  - In the 2nd column: The departure and arrival dates 

### Music DB Search

In [6]:
db_uri = 'sqlite:///resources/Chinook.db'
db = SQLDatabase.from_uri(db_uri)

print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")
print(f'Sample output: {db.run("SELECT * FROM Artist LIMIT 5;")}')

Dialect: sqlite
Available tables: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
Sample output: [(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains')]


In [7]:
toolkit = SQLDatabaseToolkit(db=db, llm=model)

db_tools = toolkit.get_tools()

for db_tool in db_tools:
    print(f"{db_tool.name}: {db_tool.description}\n")

sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.

sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3

sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!



## State

In [8]:
class WeddingState(AgentState):
    origin: str
    destination: str
    date: str
    music: str
    venue: str
    catering: str
    guest_count: str

@tool
def update_state(runtime: ToolRuntime, origin: Optional[str] = '', destination: Optional[str] = '', date: Optional[str] = '', music: Optional[str] = '', venue: Optional[str] = '', catering: Optional[str] = '', guest_count: Optional[str] = '') -> Command:
    """Update the state about the wedding plan once they've revealed it.
    Args:
    origin: where the couple is from
    destination: where the wedding will take place
    date: when the wedding will take place
    music: what type of music they want
    venue: what type of venue they want
    catering: what type of catering they want
    guest_count: how many guests they want to invite"""
    return Command(update={
        "origin": origin if origin else runtime.state.get('origin'),
        "destination": destination if destination else runtime.state.get('destination'),
        "date": date if date else runtime.state.get('date'),
        "music": music if music else runtime.state.get('music'),
        "venue": venue if venue else runtime.state.get('venue'),
        "catering": catering if catering else runtime.state.get('catering'),
        "guest_count": guest_count if guest_count else runtime.state.get('guest_count'),
        "messages": [ToolMessage("Successfully updated state", tool_call_id=runtime.tool_call_id)]}
        )

@tool
def read_state(runtime: ToolRuntime, keys: list[str]) -> dict:
    """Read the current state about the wedding plan.
    Args:
        keys: list of keys to read from the state (origin, destination, music, venue, catering, guest_count)
    """
    try:
        return {key: runtime.state.get(key, f'Key "{key}" Not Found') for key in keys}
    except Exception as e:
        return f"Error reading state: {e}"

## Agents

### Sub Agents

In [9]:
venue_agent = create_agent(
    model=model,
    tools=[web_search],
    checkpointer=InMemorySaver(),
    system_prompt="""
    You are a venue specialist. Search for venues in the desired location, and with the desired capacity.
    You are not allowed to ask any more follow up questions, you must find the best venue options based on the following criteria:
    - Price (lowest)
    - Capacity (exact match)
    - Reviews (highest)
    You may need to make multiple searches to iteratively find the best options.
    If it is getting difficult to find options, try to relax the criteria a bit and find good options that are close to the criteria. Do not perfom more than 20 searches.
"""
)

catering_agent = create_agent(
    model=model,
    tools=[web_search],
    checkpointer=InMemorySaver(),
    system_prompt="""
You are a Wedding Catering Specialist Agent. Search for up to 3 vendors matching the requirements requests (guest_count, cuisine, and budget...)
You are not allowed to ask any more follow up questions, you must find the best catering options based on the following criteria:
- Price (lowest)
- Capacity (exact match)
- Reviews (highest)
"""
)

playlist_agent = create_agent(
    model=model,
    tools=[*db_tools, web_search],
    checkpointer=InMemorySaver(),
    system_prompt="""
    You are a playlist specialist. Query the sql database and curate the perfect playlist for a wedding given a genre.
    Once you have your playlist, calculate the total duration and cost of the playlist, each song has an associated price.
    If you run into errors when querying the database, try to fix them by making changes to the query.
    Do not come back empty handed, keep trying to query the db until you find a list of songs.
    You may need to make multiple queries to iteratively find the best options.
"""
)

travel_agent = create_agent(
    model=model,
    tools=[travel_tools[0]],
    checkpointer=InMemorySaver(),
    system_prompt="""
    You are a travel agent. Search for flights to the desired destination wedding location.
    You are not allowed to ask any more follow up questions, you must find the best flight options based on the following criteria:
    - Price (lowest, economy class)
    - Duration (shortest)
    - Date (time of year which you believe is best for a wedding at this location)
    To make things easy, only look for one ticket, one way.
    You may need to make multiple searches to iteratively find the best options.
    You will be given no extra information, only the origin and destination. It is your job to think critically about the best options.
    Once you have found the best options, let the user know your shortlist of options.
"""
)

In [10]:
@tool
def call_venue_agent(runtime: ToolRuntime) -> str:
    """Call the venue agent to get venue recommendations based on the current state."""
    destination = runtime.state.get("destination", "")
    guest_count = runtime.state.get("guest_count", "")
    date = runtime.state.get("date", "")
    music = runtime.state.get('music', 'No specific music preference')

    if not destination or not guest_count:
        return "Error: destination and guest_count must be provided in the state first."

    query = f"""Search wedding venues considering
    - Place: {destination}
    - Guest Count: {guest_count}
    - Date: {date}.
    Optional considerations:
    - Music: {music}"""
    result = venue_agent.invoke({"messages": [HumanMessage(content=query)]})

    return result["messages"][-1].content

@tool
def call_catering_agent(runtime: ToolRuntime) -> str:
    """Call the catering agent to get catering recommendations based on the current state."""
    destination = runtime.state.get("destination", "")
    guest_count = runtime.state.get("guest_count", "")
    date = runtime.state.get("date", "")
    music = runtime.state.get('music', 'No specific music preference')

    if not destination or not guest_count:
        return "Error: destination and guest_count must be provided in the state first."

    query = f"""Search wedding catering considering
    - Place: {destination}
    - Guest Count: {guest_count}
    - Date: {date}.
    Optional considerations:
    - Music: {music}"""

    result = catering_agent.invoke({"messages": [HumanMessage(content=query)]})

    return result["messages"][-1].content

@tool
def call_playlist_agent(runtime: ToolRuntime) -> str:
    """Call the playlist agent to get playlist recommendations based on the current state."""
    music = runtime.state.get('music', 'No specific music preference')

    query = f"""Search wedding playlist considering
    - Music Preferences: {music}"""

    result = playlist_agent.invoke({"messages": [HumanMessage(content=query)]})
    return result["messages"][-1].content

@tool
async def call_travel_agent(runtime: ToolRuntime) -> str:
    """Call the travel agent to get travel recommendations based on the current state."""
    origin = runtime.state.get("origin", "")
    destination = runtime.state.get("destination", "")
    date = runtime.state.get("date", "")
    music = runtime.state.get('music', 'No specific music preference')

    if not origin or not destination:
        return "Error: origin and destination must be provided in the state first."

    query = f"""Search wedding travel considering
    - Origin: {origin}
    - Destination: {destination}
    - Date: {date}.
    Optional considerations:
    - Music: {music}"""

    result = await travel_agent.ainvoke({"messages": [HumanMessage(content=query)]})

    return result["messages"][-1].content

### Main Agent

In [11]:
main_agent = create_agent(
    model=model,
    tools=[update_state, web_search, call_venue_agent, call_catering_agent, call_playlist_agent, call_travel_agent],
    checkpointer=InMemorySaver(),
    state_schema=WeddingState,
    system_prompt="""
You are a wedding coordinator. Your primary task is to gather all wedding information and update the state FIRST, then delegate tasks to your specialists.

CRITICAL: You MUST follow this exact sequence:
1. Extract from the user input: origin, destination, date, music preferences, venue preferences, catering preferences, and guest count
2. Call update_state with all the information you gathered (use empty strings for missing info)
3. ONLY AFTER the state is updated, call the specialist agents (venue, catering, playlist, travel)
4. Coordinate and present the final recommendations

Do not attempt to call specialist agents (call_venue_agent, call_catering_agent, call_playlist_agent, call_travel_agent) until you have successfully updated the state.
If some information is missing and not critical, still update the state with what you have and proceed.
Once you have received their answers, coordinate the perfect wedding and present your recommendations.
"""
)


In [12]:
import threading
input = """I want to plan a wedding for 100 guests with a budget of $20,000. The wedding will be in New York on July, but we are based in Chicago.
We want a modern style venue, Italian catering, and a mix of pop and jazz music.
Can you help us plan this?"""
config = {"configurable": {"thread_id": threading.get_ident()}}

response = await main_agent.ainvoke({'messages': [HumanMessage(content=input)]}, config=config)

In [ ]:
response['messages'][-1].content

'Wonderful—we’ve started strong. Here’s what I’ve set up and what I’ve gathered from our specialist tools. I’ll keep you in the loop and present concrete options you can pick from, then I’ll coordinate final planning.\n\nWhat I’ve captured and updated in the plan\n- Origin: Chicago\n- Destination: New York\n- Date: July (please provide the exact date in YYYY-MM-DD when you’re ready)\n- Music preference: Mix of Pop and Jazz\n- Venue preference: Modern style venue\n- Catering preference: Italian catering\n- Guest count: 100\n\nWhat the specialists produced (summary of options and next steps)\n1) Venue options (modern, 100-capacity, NYC area)\n- LOFT39 – New York, NY\n  - Capacity: 100\n  - Price: from $3,800\n  - Why it fits: Modern loft vibe, strong value, flexible layout; good for a stylish July event\n  - Considerations: Check sound policies and exact package details for 100 guests\n\n- Stand Up NY – New York, NY (Upper West Side)\n  - Capacity: Up to 100\n  - Price: Starting around $